In [41]:
import pandas as pd
import mysql.connector
from datetime import datetime

In [42]:
columns_to_read = ['title', 'authors', 'isbn', 'language_code', 'num_pages', 'publication_date', 'publisher']

df = pd.read_csv("../data/books.csv", usecols = columns_to_read)
valid_rows = []
for index, row in df.iterrows():
    try:
        pd.to_datetime(row['publication_date'], format='%m/%d/%Y')
        valid_rows.append(row)
    except ValueError:
        print(f"Skipping row {index} due to failed date conversion: {row['publication_date']}")

df = pd.DataFrame(valid_rows)
df['publication_date'] = pd.to_datetime(df['publication_date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

Skipping row 3348 due to failed date conversion: 6
Skipping row 4702 due to failed date conversion: 4
Skipping row 5877 due to failed date conversion: 4
Skipping row 8180 due to failed date conversion: 11/31/2000
Skipping row 8979 due to failed date conversion: 0
Skipping row 11098 due to failed date conversion: 6/31/1982


In [43]:
# Dictionary mapping language codes to full names
language_map = {
    'eng': 'English',
    'en-US': 'English (United States)',
    'fre': 'French',
    'spa': 'Spanish',
    'en-GB': 'English (United Kingdom)',
    'mul': 'Multiple languages',
    'grc': 'Ancient Greek',
    'enm': 'Middle English',
    'en-CA': 'English (Canada)',
    'ger': 'German',
    'jpn': 'Japanese',
    'ara': 'Arabic',
    'nl': 'Dutch',
    'zho': 'Chinese',
    'lat': 'Latin',
    'por': 'Portuguese',
    'srp': 'Serbian',
    'ita': 'Italian',
    'rus': 'Russian',
    'msa': 'Malay',
    'glg': 'Galician',
    'wel': 'Welsh',
    'swe': 'Swedish',
    'nor': 'Norwegian',
    'tur': 'Turkish',
    'gla': 'Gaelic',
    'ale': 'Aleut'
}

# Replace language codes with full names
df['language_code'] = df['language_code'].map(language_map)

# Insert Functions

In [44]:
def insert_author(cursor, name):
    query = "SELECT id FROM author WHERE name = %s"
    cursor.execute(query, (name, ))
    author_data = cursor.fetchone()
    if author_data is None:
        query = "INSERT INTO author (name) VALUES (%s)"
        cursor.execute(query, (name, ))
        return cursor.lastrowid
    else:
        return author_data[0]

def insert_publisher(cursor, publisher_name):
    query = "SELECT id FROM publisher WHERE name = %s"
    cursor.execute(query, (publisher_name,))
    pub_data = cursor.fetchone()
    if pub_data is None:
        query = "INSERT INTO publisher (name) VALUES (%s)"
        cursor.execute(query, (publisher_name,))
        return cursor.lastrowid
    else:
        return pub_data[0]

def insert_language(cursor, language_name):
    query = "SELECT id FROM language WHERE name = %s"
    cursor.execute(query, (language_name,))
    lang_data = cursor.fetchone()
    if lang_data is None:
        query = "INSERT INTO language (name) VALUES (%s)"
        cursor.execute(query, (language_name,))
        return cursor.lastrowid
    else:
        return lang_data[0]
    
def insert_book(cursor, title, isbn, language_code, num_pages, publication_date, publisher_id):
    query = "INSERT INTO book (title, isbn, language_id, num_pages, publication_date, publisher_id) VALUES (%s, %s, %s, %s, %s, %s)"
    cursor.execute(query, (title, isbn, language_code, num_pages, publication_date, publisher_id))


def insert_book_author(cursor, author_id, book_id):
    query = "SELECT author_id, book_id FROM book_author WHERE author_id = %s AND book_id = %s"
    cursor.execute(query, (author_id, book_id))
    book_author_data = cursor.fetchone()
    if book_author_data is None:
        query = "INSERT INTO book_author (author_id, book_id) VALUES (%s, %s)"
        cursor.execute(query, (author_id, book_id))
        return cursor.lastrowid
    else:
        return book_author_data[0]

In [45]:
# Connect to the MySQL database
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="booktrackerdb"
)
cursor = connection.cursor()


for idx, row in df.iterrows():
    # Insert authors
    authors = row["authors"].split("/")
    for author in authors:
        insert_author(cursor, author)

    # Insert publisher and language
    insert_publisher(cursor, row["publisher"].strip())
    insert_language(cursor, row['language_code'].strip())
    connection.commit()  # Commit changes to ensure the IDs are available for the next insert
    

    
    # Get publisher ID
    query = "SELECT id FROM publisher WHERE name = %s"
    cursor.execute(query, (row["publisher"],))
    publisher_id = cursor.fetchone()[0]

    # Get language ID
    query = "SELECT id FROM language WHERE name = %s"
    cursor.execute(query, (row["language_code"],))
    language_id = cursor.fetchone()[0]

    # Insert book data
    insert_book(cursor, row["title"], row["isbn"], language_id, row["num_pages"], row["publication_date"], publisher_id)
    connection.commit()

    # Get book ID
    query = "SELECT id FROM book WHERE isbn = %s"
    cursor.execute(query, (row["isbn"],))
    book_id = cursor.fetchone()[0]

    # Get author IDs
    author_ids = []
    for author in authors:
        print(author)
        query = "SELECT id FROM author WHERE name = %s"
        cursor.execute(query, (author, ))
        author_id = cursor.fetchone()[0]
        author_ids.append(author_id)

    # Insert book_author data
    for author_id in author_ids:
        insert_book_author(cursor, author_id, book_id)
        

    
    
    connection.commit()

J.K. Rowling
Mary GrandPré
J.K. Rowling
Mary GrandPré
J.K. Rowling
J.K. Rowling
Mary GrandPré
J.K. Rowling
Mary GrandPré
W. Frederick Zimmerman
J.K. Rowling
Douglas Adams
Douglas Adams
Douglas Adams
Douglas Adams
Stephen Fry
Douglas Adams
Bill Bryson
Bill Bryson
Bill Bryson
Bill Bryson
Bill Bryson
Bill Bryson
Bill Bryson
Bill Bryson
Bill Bryson
J.R.R. Tolkien
J.R.R. Tolkien
J.R.R. Tolkien
J.R.R. Tolkien
Alan  Lee
Chris   Smith
Christopher  Lee
Richard Taylor
Jude Fisher
Dave Thomas
David Heinemeier Hansson
Leon Breedt
Mike Clark
Thomas  Fuchs
Andreas  Schwarz
Gary Paulsen
Donna Ickes
Edward Sciranko
Keith Vasconcelles
Gary Paulsen
Molly Hatchet
Dale Peck
Angela Knight
Sahara Kelly
Judy Mays
Marteeka Karland
Kate Douglas
Shelby Morgen
Lacey Savage
Kate Hill
Willa Okati
Delia Sherman
Patricia A. McKillip
Zilpha Keatley Snyder
Kate Horsley
Philippa Carr
Edward P. Jones
Edward P. Jones
Kevin R. Free
Edward P. Jones
Satyajit Das
Mark Smylie
John McPhee
William Howarth
John McPhee
John McPhe

In [40]:
cursor.close()
connection.close()